# Notebook 7 – Prompt Engineering con LLM (Groq + Llama 3.x)

**Objetivo (35 minutos):**  
Trabajar técnicas de *prompt engineering* usando un LLM real, aplicadas a SISRED:

- Zero-shot  
- Few-shot  
- Cadena de pensamiento (Chain-of-Thought)  
- Control de estilo / rol  
- Reducción de alucinaciones  
- Plantillas de prompt aplicadas a procesos institucionales  

Al final, cada participante diseñará **2 prompts productivos** para su proyecto integrador.

💡 **LLM usado en este notebook:**  
Usaremos la API de **Groq Cloud** con un modelo tipo *Llama 3.x Instruct* (endpoint compatible con OpenAI).

> Antes de ejecutar, configura la variable de entorno `GROQ_API_KEY` en tu sistema o en el entorno del notebook.



## 1. ¿Qué es Prompt Engineering?

Prompt Engineering es el conjunto de técnicas diseñadas para:

1. Aumentar precisión
2. Reducir alucinaciones
3. Forzar estructura
4. Controlar estilo, tono y rol
5. Hacer que el modelo razone en pasos

En esta sesión trabajaremos *directamente* con ejemplos aplicados a procesos reales de SISRED.


In [ ]:
# ============================================================
# Celda 2 – Configurar API de Groq (Llama 3.x)
# ============================================================

import requests
import os



GROQ_API_KEY = "TU_API_KEY_AQUI"  # Reemplaza con tu clave real

# No continuar si no está configurada
if GROQ_API_KEY.strip() == "" or "AQUI" in GROQ_API_KEY:
    raise ValueError("Debes ingresar tu GROQ_API_KEY antes de usar el notebook.")

GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"
GROQ_MODEL = "llama-3.1-8b-instant"   


def llm_chat(
    user_message: str,
    system_message: str = "Eres un asistente experto en SISRED, seguridad, despliegues y análisis técnico.",
    temperature: float = 0.2,
    max_tokens: int = 512
):
    """
    Función simple para consultar un LLM real usando la API de Groq.
    """
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": GROQ_MODEL,
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message},
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
    }

    response = requests.post(GROQ_API_URL, json=payload, headers=headers)
    response.raise_for_status()

    return response.json()["choices"][0]["message"]["content"]


## 1. Roadmap del notebook (35 min)

Hoy veremos con ejemplos reales usando un LLM:

1. Zero-shot prompting  
2. Few-shot prompting  
3. Cadena de pensamiento  
4. Control de estilo / rol  
5. Técnicas para reducir alucinaciones  
6. Plantillas aplicables a SISRED  
7. Mini-desafío final  


## 2. Zero-shot prompting

Prompting sin ejemplos: solo una instrucción clara.

Situación típica:
- Redacción de políticas
- Resumen de logs
- Explicar un flujo institucional


In [4]:
prompt_zero = """
Explica el flujo básico para levantar un incidente en un sistema institucional similar a SISRED.
"""

print(llm_chat(prompt_zero))


El flujo básico para levantar un incidente en un sistema institucional similar a SISRED puede variar dependiendo de las políticas y procedimientos de la institución, pero a continuación te presento un flujo general:

**1. Identificación del incidente**

* Un usuario o un sistema detecta un problema o una anomalía en el sistema.
* El problema puede ser una falla en el sistema, una vulnerabilidad, un ataque cibernético, un error de configuración, etc.

**2. Notificación**

* El usuario o el sistema notifica al equipo de soporte técnico o al equipo de seguridad de la institución.
* La notificación puede ser a través de un sistema de gestión de incidentes (SIEM), un correo electrónico, un mensaje de texto, etc.

**3. Evaluación inicial**

* El equipo de soporte técnico o el equipo de seguridad evalúa la gravedad del incidente y determina si es necesario activar el proceso de incidente.
* Se evalúa la posible afectación al sistema, la privacidad de los usuarios y la seguridad de la informac

## 3. Few-shot prompting

Le damos al modelo **ejemplos previos** para enseñarle un formato.

Ideal para:
- Clasificar incidentes
- Estandarizar reportes
- Convertir logs a un formato oficial


In [5]:
prompt_few = """
Actúa como analista de operaciones de SISRED.
Clasifica el impacto de incidentes según el formato:
Descripción → Impacto

Ejemplos:
"Caída total del portal principal" → Alto
"Retraso de 5s en módulo secundario" → Bajo
"Intermitencia en notificaciones" → Medio

Clasifica:
"Demora de 20s en autenticación para el 30% de los usuarios"
"Error gráfico en reportes, sin afectar datos"
"""

print(llm_chat(prompt_few))


Como analista de operaciones de SISRED, clasifico los incidentes según el formato solicitado:

"Demora de 20s en autenticación para el 30% de los usuarios" → Medio
La demora en autenticación puede afectar la experiencia del usuario y la productividad, pero no es una caída total del servicio ni un error crítico que afecte la disponibilidad del sistema.

"Error gráfico en reportes, sin afectar datos" → Bajo
El error gráfico en reportes no afecta la integridad de los datos ni la disponibilidad del sistema, por lo que se considera un incidente de impacto bajo.


## 4. Cadena de pensamiento (Chain-of-Thought)

Forzamos al LLM a **razonar paso a paso** antes de responder.

Patrón:
> “Razona paso a paso y luego entrega la conclusión.”


In [6]:
prompt_cot = """
Queremos determinar si un incidente es crítico.

Criterios:
- Afecta autenticación → crítico
- Afecta disponibilidad general → crítico
- Más de 30 usuarios impactados → crítico

Caso:
"Usuarios no pueden autenticarse y backend devuelve error 500."

Instrucción:
Razona paso a paso y luego da la clasificación final.
"""

print(llm_chat(prompt_cot, temperature=0.1))


Para determinar si el incidente es crítico, debemos analizar los criterios establecidos y aplicarlos al caso presentado.

**Paso 1: Afecta autenticación**
- El caso menciona que "Usuarios no pueden autenticarse". Esto significa que el incidente afecta directamente la autenticación de los usuarios.
- Según el criterio, si afecta autenticación, el incidente es crítico.

**Paso 2: Afecta disponibilidad general**
- El caso menciona que el backend devuelve error 500, lo que sugiere que el incidente afecta la disponibilidad del sistema.
- Sin embargo, no se menciona que la disponibilidad general esté afectada, solo que el backend devuelve un error.
- Según el criterio, si afecta disponibilidad general, el incidente es crítico. Pero en este caso, no hay suficiente información para determinar si la disponibilidad general está afectada.

**Paso 3: Más de 30 usuarios impactados**
- El caso no proporciona información sobre el número de usuarios impactados.
- Según el criterio, si más de 30 usuari

## 5. Control de estilo / rol

Podemos definir el “rol institucional” del modelo:

- Analista SISRED
- Ingeniero de despliegue
- Experto en ciberseguridad
- Redactor técnico del BCN

Esto hace las respuestas consistentes y realistas.


In [7]:
system_sisred = """
Eres un analista senior de operaciones de SISRED.
Escribes con tono técnico, claro, preciso y formal.
"""

prompt_rol = """
Describe el procedimiento estándar para ejecutar un rollback controlado
de una versión reciente desplegada en producción.
"""

print(llm_chat(prompt_rol, system_message=system_sisred))



**Procedimiento de Rollback Controlado**

**Objetivo:** Realizar un rollback controlado de una versión reciente desplegada en producción, minimizando el impacto en los usuarios y garantizando la integridad de los datos.

**Pre-requisitos:**

1. Se debe tener acceso a la plataforma de gestión de versiones (Git, SVN, etc.) y a la herramienta de despliegue (Ansible, Puppet, etc.).
2. Se debe tener conocimiento de la estructura de la aplicación y de los componentes involucrados en el despliegue.
3. Se debe tener acceso a la base de datos y a la plataforma de monitoreo (Prometheus, Grafana, etc.).

**Pasos del Procedimiento:**

**1. Identificar la versión a rollback**

* Verificar la versión actual desplegada en producción y determinar la versión anterior que se desea desplegar.
* Verificar la historia de cambios en la plataforma de gestión de versiones para asegurarse de que la versión anterior no tenga conflictos con la versión actual.

**2. Crear un entorno de pruebas**

* Crear un entor

## 6. Reducir alucinaciones

Técnicas:

1. Forzar al modelo a usar **solo** un contexto dado.  
2. Autorizar explícitamente "no lo sé" si la info no aparece.  
3. Especificar restricciones fuertes.

Simularemos un contexto reducido.


In [8]:
contexto = """
Contexto disponible:

La política de seguridad exige 2FA para acceso a producción.
Se prohíbe compartir credenciales.
Los incidentes deben reportarse en menos de 2 horas.
"""

prompt_no_hall = f"""
{contexto}

Instrucción:
Responde solo usando el contexto anterior.
Si la información no aparece, di: "No está especificado en el contexto."

Pregunta:
¿Cuánto duran los tokens de acceso a producción?
"""

print(llm_chat(prompt_no_hall, temperature=0.0))


No está especificado en el contexto.


## 7. Plantillas de prompt aplicadas a SISRED

Diseñamos prompts reutilizables para:

- Resúmenes técnicos
- Análisis de incidentes
- Reportes institucionales


In [9]:
plantilla_incidente = """
[ROL: Analista de operaciones SISRED]
[OBJETIVO: Generar un resumen técnico-ejecutivo de un incidente]
[FORMATO:
- Descripción breve
- Impacto
- Análisis
- Causa probable
- Próximos pasos
]

Descripción del incidente:
"{descripcion}"

Genera el resumen siguiendo el formato.
"""

ejemplo = """
Los usuarios externos no pueden acceder al portal desde las 08:15. 
Balanceador muestra errores intermitentes al conectar con backend.
"""

prompt_plantilla = plantilla_incidente.format(descripcion=ejemplo)
print(llm_chat(prompt_plantilla))


**Resumen Técnico-Ejecutivo del Incidente**

**Descripción Breve:**
El portal de acceso externo ha experimentado una interrupción en la conectividad desde las 08:15, afectando la capacidad de los usuarios externos para acceder al sistema. El balanceador de carga ha reportado errores intermitentes al conectar con el backend.

**Impacto:**
La interrupción del acceso externo ha causado una disminución en la disponibilidad del servicio, lo que puede afectar la productividad y la satisfacción del cliente. El impacto económico y de reputación puede ser significativo si no se resuelve rápidamente.

**Análisis:**
La combinación de errores intermitentes en el balanceador de carga y la inaccesibilidad del portal sugiere un problema de conectividad o de comunicación entre los componentes del sistema. Es posible que se trate de un problema de configuración, un error de hardware o un problema de software en el balanceador o en el backend.

**Causa Probable:**
La causa probable del incidente es un p

## 8. Mini-desafío (3 minutos, rápido)

Diseña **2 prompts productivos** para tu proyecto integrador:

1. Un **prompt zero-shot** que automatice una tarea real (resumen, checklist, reporte).
2. Un **prompt few-shot** que enseñe un formato fijo (clasificación, extracción, normalización).

Requisitos:
- Incluye un rol institucional.
- Especifica formato.
- Evita alucinaciones pidiendo que cite el contexto.


In [10]:
# ============================================================
# Celda 17 – Espacio para que pruebes tus propios prompts
# ============================================================

mis_prompts = [
    "Escribe aquí tu prompt zero-shot de tu proyecto.",
    "Escribe aquí tu prompt few-shot de tu proyecto."
]

for p in mis_prompts:
    print("\n" + "="*80)
    print("PROMPT:\n", p)
    print("\nRESPUESTA:\n")
    print(llm_chat(p))



PROMPT:
 Escribe aquí tu prompt zero-shot de tu proyecto.

RESPUESTA:

**Prompt Zero-Shot:**

"Desarrolla un modelo de asistencia experta en SISRED (Seguridad, Informática, Sistemas, Redes, Despliegues y Análisis Técnico) que pueda responder a preguntas y proporcionar información sobre:

- Seguridad informática: vulnerabilidades, amenazas, protección de datos, autenticación y autorización.
- Despliegues de sistemas: planificación, implementación, pruebas y monitoreo de sistemas informáticos.
- Análisis técnico: diagnóstico de problemas, optimización de rendimiento, análisis de tráfico de red y seguridad de la información.
- Redes y comunicaciones: protocolos de red, configuración de routers y switches, seguridad de la red y optimización de la infraestructura de red.
- Informática: sistemas operativos, bases de datos, programación y desarrollo de software.
- Técnicas de seguridad: criptografía, firewalls, intrusion detection y prevention systems (IDPS), y seguridad de la información.

